In [1]:
# ===========================================
# ✅ LANGGRAPH + LANGCHAIN + PINECONE ENV SETUP (Nov 2025)
# ===========================================
!pip install -U pip setuptools wheel

# ---- Core GenAI Stack ----
!pip install -qU \
  langgraph langchain langchain-core langchain-community \
  langchainhub langchain-huggingface \
  tiktoken huggingface-hub

# ---- Vector DBs / Embeddings ----
!pip install -qU \
  pinecone \
  chromadb qdrant-client[grpc] sentence-transformers

# ---- Optional Observability / Utils ----
!pip install -qU langsmith-client langfuse rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.17.0 requires opentelemetry-api<=1.37.0,>=1.37.0, but you have opentelemetry-api 1.38.0 which is incompatible.
google-adk 1.17.0 requires opentelemetry-sdk<=1.37.0,>=1.37.0, but you have opentelemetry-sdk 1.38.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.37.0 requires opentelemetry-ex

In [2]:
import torch, importlib
from importlib.metadata import version, PackageNotFoundError
from rich.console import Console

console = Console()

console.rule("[bold cyan]LangGraph Stack Diagnostic (Nov 2025)")
pkgs = [
    "langgraph",
    "langchain",
    "langchain-core",
    "langchain-community",
    "langchainhub",
    "pinecone",
    "huggingface_hub",
    "tiktoken",
    "sentence_transformers",
    "chromadb",
    "qdrant_client",
]

for pkg in pkgs:
    try:
        console.print(f"✅ {pkg} [green]{version(pkg)}[/green]")
    except PackageNotFoundError:
        console.print(f"❌ {pkg} not installed")

# ---- GPU Check ----
gpu_available = torch.cuda.is_available()
console.rule("[bold yellow]System")
console.print(f"GPU Available: [bold green]{gpu_available}[/bold green]" if gpu_available else "[bold red]GPU not detected[/bold red]")

# ---- LangGraph Core Check ----
try:
    from langgraph.graph import Graph, START, END, node

    @node
    def greet(name: str) -> str:
        return f"Hello, {name} 👋"

    with Graph() as g:
        greet(START) >> END

    app = g.compile()
    result = app.invoke("LangGraph")
    console.print(f"[bold green]LangGraph operational test passed:[/bold green] {result}")
except Exception as e:
    console.print(f"[bold red]LangGraph test failed:[/bold red] {e}")

────────────────────────────────────── LangGraph Stack Diagnostic (Nov 2025) ──────────────────────────────────────

✅ langgraph 1.0.2

✅ langchain 1.0.3

✅ langchain-core 1.0.3

✅ langchain-community 0.4.1

...

✅ chromadb 1.3.4

✅ qdrant_client 1.15.1

───────────────────────────────────────────────────── System ──────────────────────────────────────────────────────

GPU not detected

LangGraph test failed: cannot import name 'Graph' from 'langgraph.graph' 
(/usr/local/lib/python3.12/dist-packages/langgraph/graph/__init__.py)

In [7]:
# ===========================================
# ✅ LANGGRAPH FUNDAMENTALS — Fixed for v1.2+
# ===========================================

from typing import TypedDict
from langgraph.graph import StateGraph

# 1️⃣ Define your state schema
class MyState(TypedDict, total=False):
    name: str
    greeting: str
    compliment: str
    farewell: str

# 2️⃣ Define simple workflow nodes
def greet(state: MyState) -> MyState:
    name = state.get("name", "LangGraph User")
    state["greeting"] = f"Hello, {name} 👋"
    return state

def compliment(state: MyState) -> MyState:
    state["compliment"] = f"{state['greeting']} — You're awesome! 🚀"
    return state

def farewell(state: MyState) -> MyState:
    state["farewell"] = f"{state['compliment']}\nGoodbye from LangGraph 🌙"
    return state

# 3️⃣ Build the graph
graph = StateGraph(MyState)

graph.add_node("greet", greet)
graph.add_node("compliment", compliment)
graph.add_node("farewell", farewell)

graph.set_entry_point("greet")
graph.add_edge("greet", "compliment")
graph.add_edge("compliment", "farewell")
graph.set_finish_point("farewell")

# 4️⃣ Compile & Run
app = graph.compile()

result = app.invoke({"name": "Priyam"})
print("🧩 Graph Execution Output:\n")
print(result["farewell"])

🧩 Graph Execution Output:

Hello, Priyam 👋 — You're awesome! 🚀
Goodbye from LangGraph 🌙


In [9]:
# ===========================================
# ✅ CELL 4 — LangGraph Environment Diagnostic + Graph Introspection (v1.2+)
# Compatible with LangGraph 1.2.x–1.3.x (Nov 2025)
# ===========================================

import importlib.metadata
import sys, platform, psutil
from typing import TypedDict
from langgraph.graph import StateGraph

# ---- 1️⃣ ENVIRONMENT REPORT ----
print("🔍 LANGGRAPH ENVIRONMENT DIAGNOSTIC\n" + "="*45)

def version_of(pkg):
    try:
        return importlib.metadata.version(pkg)
    except importlib.metadata.PackageNotFoundError:
        return "❌ Not Installed"

pkgs = [
    "langgraph", "langchain", "langchain-core",
    "langchain-community", "huggingface_hub",
    "tiktoken", "pinecone"
]

for p in pkgs:
    print(f"{p:22}: {version_of(p)}")

print("\n🧠 Python:", sys.version.split()[0])
print("💻 Platform:", platform.platform())
print(f"🧩 CPU cores: {psutil.cpu_count(logical=True)}")
print(f"🧠 Memory: {round(psutil.virtual_memory().total / 1e9, 2)} GB")
print("="*45, "\n")

# ---- 2️⃣ BUILD TEST GRAPH ----
class MyState(TypedDict, total=False):
    name: str
    greeting: str
    compliment: str
    farewell: str

def greet(state: MyState) -> MyState:
    name = state.get("name", "LangGraph User")
    state["greeting"] = f"Hello, {name} 👋"
    return state

def compliment(state: MyState) -> MyState:
    state["compliment"] = f"{state['greeting']} — You're awesome! 🚀"
    return state

def farewell(state: MyState) -> MyState:
    state["farewell"] = f"{state['compliment']}\nGoodbye from LangGraph 🌙"
    return state

graph = StateGraph(MyState)
graph.add_node("greet", greet)
graph.add_node("compliment", compliment)
graph.add_node("farewell", farewell)

graph.set_entry_point("greet")
graph.add_edge("greet", "compliment")
graph.add_edge("compliment", "farewell")
graph.set_finish_point("farewell")

app = graph.compile()

# ---- 3️⃣ EXECUTION TEST ----
print("🚀 Testing graph execution...\n")
result = app.invoke({"name": "Priyam"})
print("✅ Graph ran successfully!\n")
print("Output Preview:\n----------------")
print(result["farewell"])
print("\n")

# ---- 4️⃣ GRAPH STRUCTURE INSPECTION ----
print("🧩 GRAPH STRUCTURE\n" + "="*45)
print("Nodes:")
for n in graph.nodes:
    print(" -", n)

print("\nEdges:")
for e in graph.edges:
    print(" -", e[0], "→", e[1])

# ✅ Access entry/finish points safely from compiled app.config
cfg = getattr(app, "config", {})
entry = getattr(cfg, "entry_point", None)
finish = getattr(cfg, "finish_point", None)
print("\nEntry Point:", entry or "⚠️ Unavailable in this build")
print("Finish Point:", finish or "⚠️ Unavailable in this build")
print("="*45)

🔍 LANGGRAPH ENVIRONMENT DIAGNOSTIC
langgraph             : 1.0.2
langchain             : 1.0.3
langchain-core        : 1.0.3
langchain-community   : 0.4.1
huggingface_hub       : 0.36.0
tiktoken              : 0.12.0
pinecone              : 7.3.0

🧠 Python: 3.12.12
💻 Platform: Linux-6.6.105+-x86_64-with-glibc2.35
🧩 CPU cores: 2
🧠 Memory: 13.61 GB

🚀 Testing graph execution...

✅ Graph ran successfully!

Output Preview:
----------------
Hello, Priyam 👋 — You're awesome! 🚀
Goodbye from LangGraph 🌙


🧩 GRAPH STRUCTURE
Nodes:
 - greet
 - compliment
 - farewell

Edges:
 - __start__ → greet
 - farewell → __end__
 - compliment → farewell
 - greet → compliment

Entry Point: ⚠️ Unavailable in this build
Finish Point: ⚠️ Unavailable in this build


In [10]:
# ===========================================
# ✅ CELL 5 — Conditional Logic & Dynamic Edge Traversal
# Works for LangGraph 1.2.x–1.3.x (Nov 2025)
# ===========================================

from typing import TypedDict, Literal
from langgraph.graph import StateGraph, END

# ---- 1️⃣ DEFINE STATE MODEL ----
class MoodState(TypedDict, total=False):
    name: str
    mood: Literal["happy", "sad", "neutral"]
    message: str
    final: str


# ---- 2️⃣ DEFINE NODE FUNCTIONS ----
def ask_mood(state: MoodState) -> MoodState:
    """Simulate reading user mood from input/state."""
    name = state.get("name", "Friend")
    mood = state.get("mood", "neutral")
    state["message"] = f"Hi {name}! I see you're feeling {mood} today."
    return state


def cheer_up(state: MoodState) -> MoodState:
    """Triggered if mood is sad."""
    state["final"] = f"{state['message']} 🌻 Remember, every cloud has a silver lining!"
    return state


def celebrate(state: MoodState) -> MoodState:
    """Triggered if mood is happy."""
    state["final"] = f"{state['message']} 🎉 That's awesome! Keep the good vibes going!"
    return state


def neutral_chat(state: MoodState) -> MoodState:
    """Triggered if mood is neutral."""
    state["final"] = f"{state['message']} 🤔 Let's make today interesting!"
    return state


# ---- 3️⃣ DEFINE CONDITIONAL EDGE LOGIC ----
def mood_router(state: MoodState):
    """Determine next node dynamically based on mood."""
    mood = state.get("mood", "neutral")
    if mood == "happy":
        return "celebrate"
    elif mood == "sad":
        return "cheer_up"
    else:
        return "neutral_chat"


# ---- 4️⃣ BUILD GRAPH ----
graph = StateGraph(MoodState)

graph.add_node("ask_mood", ask_mood)
graph.add_node("cheer_up", cheer_up)
graph.add_node("celebrate", celebrate)
graph.add_node("neutral_chat", neutral_chat)

graph.set_entry_point("ask_mood")
graph.add_conditional_edges("ask_mood", mood_router)
graph.add_edge("cheer_up", END)
graph.add_edge("celebrate", END)
graph.add_edge("neutral_chat", END)

app = graph.compile()


# ---- 5️⃣ EXECUTION TESTS ----
print("🚀 Conditional Graph Execution Tests\n" + "="*45)

scenarios = [
    {"name": "Priyam", "mood": "happy"},
    {"name": "Alex", "mood": "sad"},
    {"name": "Jordan", "mood": "neutral"},
    {"name": "Taylor"},  # Default fallback (neutral)
]

for case in scenarios:
    print(f"\n🧩 Input: {case}")
    output = app.invoke(case)
    print("✅ Output:", output["final"])
    print("-" * 45)

🚀 Conditional Graph Execution Tests

🧩 Input: {'name': 'Priyam', 'mood': 'happy'}
✅ Output: Hi Priyam! I see you're feeling happy today. 🎉 That's awesome! Keep the good vibes going!
---------------------------------------------

🧩 Input: {'name': 'Alex', 'mood': 'sad'}
✅ Output: Hi Alex! I see you're feeling sad today. 🌻 Remember, every cloud has a silver lining!
---------------------------------------------

🧩 Input: {'name': 'Jordan', 'mood': 'neutral'}
✅ Output: Hi Jordan! I see you're feeling neutral today. 🤔 Let's make today interesting!
---------------------------------------------

🧩 Input: {'name': 'Taylor'}
✅ Output: Hi Taylor! I see you're feeling neutral today. 🤔 Let's make today interesting!
---------------------------------------------


In [17]:
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.5 MB/s  0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [langchain-google-genai]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.9.0 which is incompatible.


In [19]:
# ==============================================
# ✅ GEMINI AUTH FIX for Colab (Nov 2025)
# ==============================================
from google.colab import userdata
import os

# 🔑 Replace with your actual Gemini API key from https://aistudio.google.com/app/apikey
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# Verify key presence
if not os.environ.get("GOOGLE_API_KEY"):
    raise ValueError("❌ Missing Gemini API key! Please set GOOGLE_API_KEY first.")

print("✅ Gemini API key configured successfully.")

✅ Gemini API key configured successfully.


In [40]:
# ===========================================
# ✅ CELL 7 — Gemini Conversational LangGraph (Final Stable)
# - Handles MAX_TOKENS cutoffs
# - Robust LLM extraction
# - Returns proper mood + reply each time
# ===========================================

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain.chat_models import init_chat_model
import time

# ---- 1️⃣ State model ----
class ChatState(TypedDict, total=False):
    messages: List[dict]
    mood: str
    reply: str

# ---- 2️⃣ LLM setup ----
def make_llm(max_tokens=512):
    return init_chat_model(
        model="gemini-2.5-flash",
        model_provider="google_genai",
        temperature=0.6,
        max_output_tokens=max_tokens,
    )

llm = make_llm()

# ---- 3️⃣ Universal extractor ----
def extract_text(result):
    if isinstance(result, str):
        return result.strip()
    if hasattr(result, "content") and result.content:
        return result.content.strip()
    if hasattr(result, "text") and result.text:
        return result.text.strip()
    if hasattr(result, "candidates"):
        try:
            c = result.candidates[0]
            if hasattr(c, "content") and getattr(c.content, "parts", None):
                parts = c.content.parts
                if parts and hasattr(parts[0], "text"):
                    return parts[0].text.strip()
        except Exception:
            pass
    if isinstance(result, dict):
        if "text" in result:
            return result["text"].strip()
    return ""

# ---- 4️⃣ Safe LLM caller with retry ----
def call_llm_safe(prompt: str, max_tokens=512):
    """Call Gemini safely with retry if truncated."""
    global llm
    result = llm.invoke(prompt)
    text = extract_text(result)
    finish_reason = ""
    if hasattr(result, "response_metadata"):
        finish_reason = result.response_metadata.get("finish_reason", "")
    # Retry if truncated or empty
    if (not text or finish_reason == "MAX_TOKENS") and max_tokens < 1024:
        print("⚠️ Gemini hit token limit, retrying with higher max_output_tokens...")
        time.sleep(0.5)
        llm = make_llm(1024)
        result = llm.invoke(prompt)
        text = extract_text(result)
    return text.strip()

# ---- 5️⃣ Node functions ----
def analyze_mood(state: ChatState) -> ChatState:
    user_text = state["messages"][-1]["content"]
    prompt = (
        f"Analyze this message and classify the user's mood as one word: "
        f"happy, sad, angry, or neutral.\n\nMessage: {user_text}"
    )
    mood_text = call_llm_safe(prompt)
    mood = "neutral"
    for m in ["happy", "sad", "angry", "neutral"]:
        if m in mood_text.lower():
            mood = m
            break
    return {**state, "mood": mood}

def respond_based_on_mood(state: ChatState) -> ChatState:
    user_text = state["messages"][-1]["content"]
    mood = state.get("mood", "neutral")
    prompt = (
        f"You are an empathetic assistant. The user feels {mood}. "
        f"Respond naturally in 1–2 sentences to: '{user_text}'."
    )
    reply = call_llm_safe(prompt)
    return {**state, "reply": reply or "[no reply generated]"}

# ---- 6️⃣ Build + compile ----
graph = StateGraph(ChatState)
graph.add_node("analyze_mood", analyze_mood)
graph.add_node("respond_based_on_mood", respond_based_on_mood)
graph.set_entry_point("analyze_mood")
graph.add_edge("analyze_mood", "respond_based_on_mood")
graph.add_edge("respond_based_on_mood", END)

interactive_app = graph.compile()

# ===========================================
# ✅ CELL 7 — Gemini Conversational LangGraph (Final Stable)
# - Handles MAX_TOKENS cutoffs
# - Robust LLM extraction
# - Returns proper mood + reply each time
# ===========================================

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain.chat_models import init_chat_model
import time

# ---- 1️⃣ State model ----
class ChatState(TypedDict, total=False):
    messages: List[dict]
    mood: str
    reply: str

# ---- 2️⃣ LLM setup ----
def make_llm(max_tokens=512):
    return init_chat_model(
        model="gemini-2.5-flash",
        model_provider="google_genai",
        temperature=0.6,
        max_output_tokens=max_tokens,
    )

llm = make_llm()

# ---- 3️⃣ Universal extractor ----
def extract_text(result):
    if isinstance(result, str):
        return result.strip()
    if hasattr(result, "content") and result.content:
        return result.content.strip()
    if hasattr(result, "text") and result.text:
        return result.text.strip()
    if hasattr(result, "candidates"):
        try:
            c = result.candidates[0]
            if hasattr(c, "content") and getattr(c.content, "parts", None):
                parts = c.content.parts
                if parts and hasattr(parts[0], "text"):
                    return parts[0].text.strip()
        except Exception:
            pass
    if isinstance(result, dict):
        if "text" in result:
            return result["text"].strip()
    return ""

# ---- 4️⃣ Safe LLM caller with retry ----
def call_llm_safe(prompt: str, max_tokens=512):
    """Call Gemini safely with retry if truncated."""
    global llm
    result = llm.invoke(prompt)
    text = extract_text(result)
    finish_reason = ""
    if hasattr(result, "response_metadata"):
        finish_reason = result.response_metadata.get("finish_reason", "")
    # Retry if truncated or empty
    if (not text or finish_reason == "MAX_TOKENS") and max_tokens < 1024:
        print("⚠️ Gemini hit token limit, retrying with higher max_output_tokens...")
        time.sleep(0.5)
        llm = make_llm(1024)
        result = llm.invoke(prompt)
        text = extract_text(result)
    return text.strip()

# ---- 5️⃣ Node functions ----
def analyze_mood(state: ChatState) -> ChatState:
    user_text = state["messages"][-1]["content"]
    prompt = (
        f"Analyze this message and classify the user's mood as one word: "
        f"happy, sad, angry, or neutral.\n\nMessage: {user_text}"
    )
    mood_text = call_llm_safe(prompt)
    mood = "neutral"
    for m in ["happy", "sad", "angry", "neutral"]:
        if m in mood_text.lower():
            mood = m
            break
    return {**state, "mood": mood}

def respond_based_on_mood(state: ChatState) -> ChatState:
    user_text = state["messages"][-1]["content"]
    mood = state.get("mood", "neutral")
    prompt = (
        f"You are an empathetic assistant. The user feels {mood}. "
        f"Respond naturally in 1–2 sentences to: '{user_text}'."
    )
    reply = call_llm_safe(prompt)
    return {**state, "reply": reply or "[no reply generated]"}

# ---- 6️⃣ Build + compile ----
graph = StateGraph(ChatState)
graph.add_node("analyze_mood", analyze_mood)
graph.add_node("respond_based_on_mood", respond_based_on_mood)
graph.set_entry_point("analyze_mood")
graph.add_edge("analyze_mood", "respond_based_on_mood")
graph.add_edge("respond_based_on_mood", END)

interactive_app = graph.compile()

# ---- 7️⃣ Interactive loop ----
print("💬 Interactive LangGraph + Gemini Chat (type 'exit' to quit)")
print("=" * 60)

state: ChatState = {"messages": []}

while True:
    user_input = input("👤 You: ").strip()
    if user_input.lower() in {"exit", "quit"}:
        print("👋 Goodbye!")
        break

    state.setdefault("messages", []).append({"role": "user", "content": user_input})
    result = interactive_app.invoke(state)
    state.update(result)

    reply = state.get("reply", "").strip()
    mood = state.get("mood", "N/A")

    # Append reply to memory for context
    state["messages"].append({"role": "assistant", "content": reply})

    print("=" * 60)
    print("🤖 LangGraph + Gemini Conversation Result")
    print("=" * 60)
    print("🧩 Mood:", mood)
    print("💬 Reply:", reply)
    print("=" * 60 + "\n")

💬 Interactive LangGraph + Gemini Chat (type 'exit' to quit)
👤 You: good day
🤖 LangGraph + Gemini Conversation Result
🧩 Mood: neutral
💬 Reply: Good day to you too! I hope you have a pleasant and productive day ahead.

👤 You: how was my day?
⚠️ Gemini hit token limit, retrying with higher max_output_tokens...
🤖 LangGraph + Gemini Conversation Result
🧩 Mood: neutral
💬 Reply: I can't tell you exactly how your day was, but I truly hope it brought you some good moments. What was it like from your perspective?



KeyboardInterrupt: Interrupted by user